In [16]:
import sys
!{sys.executable} -m pip install -r requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 25.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 20.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: Pillow
    Found existing installation: pillow 11.1.0
    Uninstalling pillow-11.1.0:
      Successfully uninstalled pillow-11.1.0

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [13]:
import torch
from diffusers import AnimateDiffPipeline, DDIMScheduler, MotionAdapter
from diffusers.utils import export_to_gif

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch_dtype = torch.float16 if device.type == "cuda" else torch.float32

# Load the motion adapter
adapter = MotionAdapter.from_pretrained("guoyww/animatediff-motion-adapter-v1-5-2", torch_dtype=torch_dtype).to(device)

# load SD 1.5 based finetuned model
#model_id = "SG161222/Realistic_Vision_V5.1_noVAE"
model_id = 'frankjoshua/toonyou_beta6'
pipe = AnimateDiffPipeline.from_pretrained(model_id, motion_adapter=adapter, torch_dtype=torch_dtype).to(device)

scheduler = DDIMScheduler.from_pretrained(
    model_id,
    subfolder="scheduler",
    clip_sample=False,
    timestep_spacing="linspace",
    beta_schedule="linear",
    steps_offset=1,
)
pipe.scheduler = scheduler

# enable memory savings
pipe.enable_vae_slicing()

if device.type == "cuda":
    pipe.enable_model_cpu_offload()

The config attributes {'motion_activation_fn': 'geglu', 'motion_attention_bias': False, 'motion_cross_attention_dim': None} were passed to MotionAdapter, but are not expected and will be ignored. Please verify your config.json configuration file.
Fetching 15 files: 100%|██████████| 15/15 [00:48<00:00,  3.20s/it]
/opt/jupyter_venv/lib/python3.10/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--frankjoshua--toonyou_beta6/snapshots/9bb5e1c5be60ab664ec80bff429126e0844ac9fe/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--frankjoshua--toonyou_beta6/snapshots/9bb5e1c5be60ab664ec80bff429126e0844ac9fe/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` 

In [23]:
def make_gif(prompt, path):
    output = pipe(
        prompt=prompt,
        negative_prompt="bad quality, worse quality",
        num_frames=16,
        guidance_scale=7.5,
        num_inference_steps=25,
        generator=torch.Generator("cpu").manual_seed(42),
    )
    frames = output.frames[0]
    return export_to_gif(frames, path)


In [25]:
make_gif("A tall tree dancing with wiggling branches in the wind, performing the moonwalk, cartoon, 2D, whimsical, colorful, cute, adorable, surprising, funny.", 
         "/workspace/animatediff/01.gif")

100%|██████████| 25/25 [00:34<00:00,  1.39s/it]


'/workspace/animatediff/01.gif'

In [26]:
prompts = ["A tall tree dancing with wiggling branches in the wind, performing the moonwalk, cartoon, 2D, whimsical, colorful, cute, adorable, surprising, funny.",
"A playful leaf flipping and twirling in the air, landing right on a squirrel’s nose, cartoon, 2D, whimsical, colorful, cute, adorable, surprising, funny.",
"A mushroom wearing a fancy oversized hat, secretly a bird’s nest, looking proud and stylish, cartoon, 2D, whimsical, colorful, cute, adorable, surprising, funny.",
"A deer playing hide-and-seek, peeking out from behind a tiny bush with antlers sticking out, cartoon, 2D, whimsical, colorful, cute, adorable, surprising, funny.",
"An owl with round glasses on its head, looking confused as it searches for them, cartoon, 2D, whimsical, colorful, cute, adorable, surprising, funny.",
"A fox with a mischievous grin, running away as bees chase him after swapping a farmer’s hat with a beehive, cartoon, 2D, whimsical, colorful, cute, adorable, surprising, funny.",
"A giant bear sleeping peacefully, snoring so loudly that nearby trees wear earmuffs, cartoon, 2D, whimsical, colorful, cute, adorable, surprising, funny.",
"A happy squirrel stuffing acorns into a mailbox while a confused mailman looks on, cartoon, 2D, whimsical, colorful, cute, adorable, surprising, funny.",
"A tiny acorn rolling downhill, bouncing straight into a sleeping bear’s ear, cartoon, 2D, whimsical, colorful, cute, adorable, surprising, funny.",
"A brave pinecone dressed as a knight, charging into battle but accidentally poking a porcupine, cartoon, 2D, whimsical, colorful, cute, adorable, surprising, funny.",
"A graceful fern dancing in the wind, suddenly getting tangled in a sticky spiderweb, cartoon, 2D, whimsical, colorful, cute, adorable, surprising, funny.",
"A lively stream flowing through the forest, happily humming, as a frog jumps in to sing along, cartoon, 2D, whimsical, colorful, cute, adorable, surprising, funny.",
"A rock sitting still, while a confused bird builds a nest on its head, cartoon, 2D, whimsical, colorful, cute, adorable, surprising, funny.",
"A juicy red berry rolling away from a hungry squirrel, hiding under a curious hedgehog, cartoon, 2D, whimsical, colorful, cute, adorable, surprising, funny.",
"A bird’s nest filled with eggs, suddenly occupied by a fluffy squirrel roommate, cartoon, 2D, whimsical, colorful, cute, adorable, surprising, funny.",
"A proud wolf howling at the moon, suddenly scared by his own echo chasing him, cartoon, 2D, whimsical, colorful, cute, adorable, surprising, funny.",
"A friendly hedgehog trying to hug a balloon, accidentally popping it, now covered in colorful confetti, cartoon, 2D, whimsical, colorful, cute, adorable, surprising, funny.",
"A tree branch waving in the wind, sending a surprised squirrel flying through the air, cartoon, 2D, whimsical, colorful, cute, adorable, surprising, funny.",
"A confident frog thinking he's a prince, kissing everything around, while mushrooms comically run away, cartoon, 2D, whimsical, colorful, cute, adorable, surprising, funny.",
"A strong tree root proudly holding up a tree, but suddenly tripping over itself, cartoon, 2D, whimsical, colorful, cute, adorable, surprising, funny.",
]
for idx, prompt in enumerate(prompts):
    print(idx)
    make_gif(prompt, '/workspace/animatediff/%02d.gif' % idx)

0


100%|██████████| 25/25 [00:34<00:00,  1.39s/it]


1


100%|██████████| 25/25 [00:35<00:00,  1.41s/it]


2


...

100%|██████████| 25/25 [00:35<00:00,  1.42s/it]


18


100%|██████████| 25/25 [00:35<00:00,  1.43s/it]


19


100%|██████████| 25/25 [00:35<00:00,  1.41s/it]


In [32]:
import torch
from diffusers import StableDiffusionUpscalePipeline
from PIL import Image
from moviepy import VideoFileClip, ImageSequenceClip
import os

# Load the upscaling model with the correct scheduler
model_id = "stabilityai/stable-diffusion-x4-upscaler"
upscaler_pipeline = StableDiffusionUpscalePipeline.from_pretrained(
    model_id, variant="fp16", torch_dtype=torch.float16
)
upscaler_pipeline.scheduler = DDIMScheduler.from_config(pipeline.scheduler.config)
upscaler_pipeline = pipeline.to("cuda")

def upscale_gif(gif_path, output_gif, factor=2, prompt="an upscaled image"):
    """
    Upscales each frame of a GIF using Stable Diffusion and saves as a new GIF.
    
    :param gif_path: Path to the input GIF file.
    :param output_gif: Path to save the upscaled GIF.
    :param prompt: Text prompt for the upscaler.
    """
    # Ensure output directory exists
    output_dir = os.path.dirname(output_gif)
    os.makedirs(output_dir, exist_ok=True)
    
    # Load GIF as video and process each frame
    clip = VideoFileClip(gif_path)
    frame_count = 0
    upscaled_frames = []
    
    for frame in clip.iter_frames(fps=clip.fps, dtype="uint8"):
        frame_image = Image.fromarray(frame).convert("RGB")
        low_res_img = frame_image.resize((clip.w * factor, clip.h * factor))
        
        # Upscale the image
        upscaled_image = upscaler_pipeline(prompt=prompt, image=low_res_img).images[0]
        upscaled_frames.append(upscaled_image)
        frame_count += 1
    
    # Save as a new GIF
    upscaled_frames = [frame.convert("RGB") for frame in upscaled_frames]
    upscaled_clip = ImageSequenceClip([frame for frame in upscaled_frames], fps=clip.fps)
    upscaled_clip.write_gif(output_gif, fps=clip.fps)
    
    print(f"Upscaled {frame_count} frames and saved GIF to {output_gif}")


Loading pipeline components...: 100%|██████████| 6/6 [00:00<00:00,  8.16it/s]
